In [1]:
import time
from pyspark.sql import SparkSession
from pyspark import StorageLevel


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1657566517605_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
def q01():
    t1 = time.time()
    sql_lineitem = spark.sql(
        """select
                l_returnflag,
                l_linestatus,
                sum(l_quantity) as sum_qty,
                sum(l_extendedprice) as sum_base_price,
                sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
                sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
                avg(l_quantity) as avg_qty,
                avg(l_extendedprice) as avg_price,
                avg(l_discount) as avg_disc,
                count(*) as count_order
            from
                lineitem
            where
                l_shipdate <= date '1998-12-01' - interval '90' day
            group by
                l_returnflag,
                l_linestatus
            order by
                l_returnflag,
                l_linestatus"""
    )

    sql_lineitem.show()
    print("Q01 Execution time (s): ", time.time() - t1)


def q02():
    t1 = time.time()
    SIZE = 15
    TYPE = "BRASS"
    REGION = "EUROPE"
    total = spark.sql(
        f"""select
                s_acctbal,
                s_name,
                n_name,
                p_partkey,
                p_mfgr,
                s_address,
                s_phone,
                s_comment
            from
                part,
                supplier,
                partsupp,
                nation,
                region
            where
                p_partkey = ps_partkey
                and s_suppkey = ps_suppkey
                and p_size = {SIZE}
                and p_type like '%{TYPE}'
                and s_nationkey = n_nationkey
                and n_regionkey = r_regionkey
                and r_name = '{REGION}'
                and ps_supplycost = (
                    select
                    min(ps_supplycost)
                    from
                    partsupp, supplier,
                    nation, region
                    where
                    p_partkey = ps_partkey
                    and s_suppkey = ps_suppkey
                    and s_nationkey = n_nationkey
                    and n_regionkey = r_regionkey
                    and r_name = '{REGION}'
                    )
            order by
                s_acctbal desc,
                n_name,
                s_name,
                p_partkey"""
    )

    total.show()
    print("Q02 Execution time (s): ", time.time() - t1)


def q03():
    t1 = time.time()
    total = spark.sql(
        """select
            l_orderkey,
            sum(l_extendedprice * (1 - l_discount)) as revenue,
            o_orderdate,
            o_shippriority
        from
            customer,
            orders,
            lineitem
        where
            c_mktsegment = 'HOUSEHOLD'
            and c_custkey = o_custkey
            and l_orderkey = o_orderkey
            and o_orderdate < date '1995-03-04'
            and l_shipdate > date '1995-03-04'
        group by
            l_orderkey,
            o_orderdate,
            o_shippriority
        order by
            revenue desc,
            o_orderdate
        limit 10"""
    )

    total.show()
    print("Q03 Execution time (s): ", time.time() - t1)


def q04():
    t1 = time.time()
    total = spark.sql(
        """select
                o_orderpriority,
                count(*) as order_count
            from
                orders
            where
                o_orderdate >= date '1993-08-01'
                and o_orderdate < date '1993-08-01' + interval '3' month
                and exists (
                    select
                        *
                    from
                        lineitem
                    where
                        l_orderkey = o_orderkey
                        and l_commitdate < l_receiptdate
                )
            group by
                o_orderpriority
            order by
                o_orderpriority"""
    )

    total.show()
    print("Q04 Execution time (s): ", time.time() - t1)


def q05():
    t1 = time.time()
    total = spark.sql(
        """select
                n_name,
                sum(l_extendedprice * (1 - l_discount)) as revenue
            from
                customer,
                orders,
                lineitem,
                supplier,
                nation,
                region
            where
                c_custkey = o_custkey
                and l_orderkey = o_orderkey
                and l_suppkey = s_suppkey
                and c_nationkey = s_nationkey
                and s_nationkey = n_nationkey
                and n_regionkey = r_regionkey
                and r_name = 'ASIA'
                and o_orderdate >= date '1996-01-01'
                and o_orderdate < date '1996-01-01' + interval '1' year
            group by
                n_name
            order by
                revenue desc"""
    )

    total.show()
    print("Q05 Execution time (s): ", time.time() - t1)


def q06():
    t1 = time.time()
    sql_lineitem = spark.sql(
        """select
                sum(l_extendedprice * l_discount) as revenue
            from
                lineitem
            where
                l_shipdate >= date '1996-01-01'
                and l_shipdate < date '1996-01-01' + interval '1' year
                and l_discount between .08 and .1
                and l_quantity < 24"""
    )
    sql_lineitem.show()
    print("Q06 Execution time (s): ", time.time() - t1)


def q07():
    t1 = time.time()
    NATION1 = "FRANCE"
    NATION2 = "GERMANY"
    total = spark.sql(
        f"""select
                supp_nation,
                cust_nation,
                l_year, sum(volume) as revenue
            from (
                select
                    n1.n_name as supp_nation,
                    n2.n_name as cust_nation,
                    extract(year from l_shipdate) as l_year,
                    l_extendedprice * (1 - l_discount) as volume
                from
                    supplier,
                    lineitem,
                    orders,
                    customer,
                    nation n1,
                    nation n2
                where
                    s_suppkey = l_suppkey
                    and o_orderkey = l_orderkey
                    and c_custkey = o_custkey
                    and s_nationkey = n1.n_nationkey
                    and c_nationkey = n2.n_nationkey
                    and (
                    (n1.n_name = '{NATION1}' and n2.n_name = '{NATION2}')
                    or (n1.n_name = '{NATION2}' and n2.n_name = '{NATION1}')
                    )
                    and l_shipdate between date '1995-01-01' and date '1996-12-31'
                ) as shipping
            group by
                supp_nation,
                cust_nation,
                l_year
            order by
                supp_nation,
                cust_nation,
                l_year"""
    )
    total.show()
    print("Q07 Execution time (s): ", time.time() - t1)


def q08():
    t1 = time.time()
    NATION = "BRAZIL"
    REGION = "AMERICA"
    TYPE = "ECONOMY ANODIZED STEEL"
    total = spark.sql(
        f"""select
                o_year,
                sum(case
                    when nation = '{NATION}'
                    then volume
                    else 0
                end) / sum(volume) as mkt_share
            from (
                select
                    extract(year from o_orderdate) as o_year,
                    l_extendedprice * (1-l_discount) as volume,
                    n2.n_name as nation
                from
                    part,
                    supplier,
                    lineitem,
                    orders,
                    customer,
                    nation n1,
                    nation n2,
                    region
                where
                    p_partkey = l_partkey
                    and s_suppkey = l_suppkey
                    and l_orderkey = o_orderkey
                    and o_custkey = c_custkey
                    and c_nationkey = n1.n_nationkey
                    and n1.n_regionkey = r_regionkey
                    and r_name = '{REGION}'
                    and s_nationkey = n2.n_nationkey
                    and o_orderdate between date '1995-01-01' and date '1996-12-31'
                    and p_type = '{TYPE}'
                ) as all_nations
            group by
                o_year
            order by
                o_year"""
    )
    total.show()
    print("Q08 Execution time (s): ", time.time() - t1)


def q09():
    t1 = time.time()
    total = spark.sql(
        """select
                nation,
                o_year,
                sum(amount) as sum_profit
            from
                (
                    select
                        n_name as nation,
                        year(o_orderdate) as o_year,
                        l_extendedprice * (1 - l_discount) - ps_supplycost * l_quantity as amount
                    from
                        part,
                        supplier,
                        lineitem,
                        partsupp,
                        orders,
                        nation
                    where
                        s_suppkey = l_suppkey
                        and ps_suppkey = l_suppkey
                        and ps_partkey = l_partkey
                        and p_partkey = l_partkey
                        and o_orderkey = l_orderkey
                        and s_nationkey = n_nationkey
                        and p_name like '%ghost%'
                ) as profit
            group by
                nation,
                o_year
            order by
                nation,
                o_year desc"""
    )
    total.show()
    print("Q09 Execution time (s): ", time.time() - t1)


def q10():
    t1 = time.time()
    total = spark.sql(
        """select
                c_custkey,
                c_name,
                sum(l_extendedprice * (1 - l_discount)) as revenue,
                c_acctbal,
                n_name,
                c_address,
                c_phone,
                c_comment
            from
                customer,
                orders,
                lineitem,
                nation
            where
                c_custkey = o_custkey
                and l_orderkey = o_orderkey
                and o_orderdate >= date '1994-11-01'
                and o_orderdate < date '1994-11-01' + interval '3' month
                and l_returnflag = 'R'
                and c_nationkey = n_nationkey
            group by
                c_custkey,
                c_name,
                c_acctbal,
                c_phone,
                n_name,
                c_address,
                c_comment
            order by
                revenue desc
            limit 20"""
    )

    total.show()
    print("Q10 Execution time (s): ", time.time() - t1)


def q11():
    t1 = time.time()
    NATION = "GERMANY"
    FRACTION = 0.0001
    total = spark.sql(
        f"""select
                ps_partkey,
                sum(ps_supplycost * ps_availqty) as value
            from
                partsupp,
                supplier,
                nation
            where
                ps_suppkey = s_suppkey
                and s_nationkey = n_nationkey
                and n_name = '{NATION}'
            group by
                ps_partkey having
                        sum(ps_supplycost * ps_availqty) > (
                    select
                        sum(ps_supplycost * ps_availqty) * {FRACTION}
                    from
                        partsupp,
                        supplier,
                        nation
                    where
                        ps_suppkey = s_suppkey
                        and s_nationkey = n_nationkey
                        and n_name = '{NATION}'
                    )
                order by
                    value desc"""
    )

    total.show()
    print("Q11 Execution time (s): ", time.time() - t1)


def q12():
    t1 = time.time()
    total = spark.sql(
        """select
                l_shipmode,
                sum(case
                    when o_orderpriority = '1-URGENT'
                        or o_orderpriority = '2-HIGH'
                        then 1
                    else 0
                end) as high_line_count,
                sum(case
                    when o_orderpriority <> '1-URGENT'
                        and o_orderpriority <> '2-HIGH'
                        then 1
                    else 0
                end) as low_line_count
            from
                orders,
                lineitem
            where
                o_orderkey = l_orderkey
                and l_shipmode in ('MAIL', 'SHIP')
                and l_commitdate < l_receiptdate
                and l_shipdate < l_commitdate
                and l_receiptdate >= date '1994-01-01'
                and l_receiptdate < date '1994-01-01' + interval '1' year
            group by
                l_shipmode
            order by
                l_shipmode"""
    )

    total.show()
    print("Q12 Execution time (s): ", time.time() - t1)


def q13():
    t1 = time.time()
    WORD1 = "special"
    WORD2 = "requests"
    total = spark.sql(
        f"""select
                c_count, count(*) as custdist
            from (
                select
                    c_custkey,
                    count(o_orderkey)
                from
                    customer left outer join orders on
                    c_custkey = o_custkey
                    and o_comment not like '%{WORD1}%{WORD2}%'
                group by
                    c_custkey
                )as c_orders (c_custkey, c_count)
            group by
                c_count
            order by
                custdist desc,
                c_count desc"""
    )
    total.show()
    print("Q13 Execution time (s): ", time.time() - t1)



def q14():
    t1 = time.time()
    total = spark.sql(
        """select
                100.00 * sum(case
                    when p_type like 'PROMO%'
                        then l_extendedprice * (1 - l_discount)
                    else 0
                end) / sum(l_extendedprice * (1 - l_discount)) as promo_revenue
            from
                lineitem,
                part
            where
                l_partkey = p_partkey
                and l_shipdate >= date '1994-03-01'
                and l_shipdate < date '1994-03-01' + interval '1' month"""
    )

    total.show()
    print("Q14 Execution time (s): ", time.time() - t1)


def q15():
    # Why `CAST` is used here?
    # From spec the l_extendedprice and l_discount are defined as decimals.
    # It defines decimal as having 12 digits and 2 digits after the point.
    # See p14 in
    # http://www.tpc.org/tpc_documents_current_versions/pdf/tpc-h_v2.17.1.pdf
    t1 = time.time()
    spark.sql(
        """create temp view revenue (supplier_no, total_revenue) as
                select
                    l_suppkey,
                    CAST(sum(l_extendedprice * (1 - l_discount)) as DECIMAL(12,2))
                from
                    lineitem
                where
                    l_shipdate >= date '1996-01-01'
                    and l_shipdate < date '1996-01-01' + interval '3' month
                group by
                    l_suppkey"""
    )
    total = spark.sql(
        """
            select
                s_suppkey,
                s_name,
                s_address,
                s_phone,
                total_revenue
            from
                supplier,
                revenue
            where
                s_suppkey = supplier_no
                and total_revenue = (
                    select
                        max(total_revenue)
                    from
                        revenue
                )
            order by
                s_suppkey"""
    )
    spark.sql(
        "drop view revenue"
    )
    total.show()
    print("Q15 Execution time (s): ", time.time() - t1)


def q16():
    t1 = time.time()
    BRAND = "Brand#45"
    TYPE = "MEDIUM POLISHED"
    SIZE1 = 49
    SIZE2 = 14
    SIZE3 = 23
    SIZE4 = 45
    SIZE5 = 19
    SIZE6 = 3
    SIZE7 = 36
    SIZE8 = 9
    total = spark.sql(
        f"""select
                p_brand,
                p_type,
                p_size,
                count(distinct ps_suppkey) as supplier_cnt
            from
                partsupp,
                part
            where
                p_partkey = ps_partkey
                and p_brand <> '{BRAND}'
                and p_type not like '{TYPE}%'
                and p_size in ({SIZE1}, {SIZE2}, {SIZE3}, {SIZE4}, {SIZE5}, {SIZE6}, {SIZE7}, {SIZE8})
                and ps_suppkey not in (
                    select
                        s_suppkey
                    from
                        supplier
                    where
                        s_comment like '%Customer%Complaints%'
                )
            group by
                p_brand,
                p_type,
                p_size
            order by
                supplier_cnt desc,
                p_brand,
                p_type,
                p_size"""
    )

    total.show()
    print("Q16 Execution time (s): ", time.time() - t1)


def q17():
    t1 = time.time()
    total = spark.sql(
        """select
                sum(l_extendedprice) / 7.0 as avg_yearly
            from
                lineitem,
                part
            where
                p_partkey = l_partkey
                and p_brand = 'Brand#23'
                and p_container = 'MED BOX'
                and l_quantity < (
                    select
                        0.2 * avg(l_quantity)
                    from
                        lineitem
                    where
                        l_partkey = p_partkey
                )"""
    )

    total.show()
    print("Q17 Execution time (s): ", time.time() - t1)


def q18():
    t1 = time.time()
    total = spark.sql(
        """select
                c_name,
                c_custkey,
                o_orderkey,
                o_orderdate,
                o_totalprice,
                sum(l_quantity)
            from
                customer,
                orders,
                lineitem
            where
                o_orderkey in (
                    select
                        l_orderkey
                    from
                        lineitem
                    group by
                        l_orderkey having
                            sum(l_quantity) > 300
                )
                and c_custkey = o_custkey
                and o_orderkey = l_orderkey
            group by
                c_name,
                c_custkey,
                o_orderkey,
                o_orderdate,
                o_totalprice
            order by
                o_totalprice desc,
                o_orderdate
            limit 100"""
    )

    total.show()
    print("Q18 Execution time (s): ", time.time() - t1)


def q19():
    t1 = time.time()
    total = spark.sql(
        """select
                sum(l_extendedprice* (1 - l_discount)) as revenue
            from
                lineitem,
                part
            where
                (
                    p_partkey = l_partkey
                    and p_brand = 'Brand#31'
                    and p_container in ('SM CASE', 'SM BOX', 'SM PACK', 'SM PKG')
                    and l_quantity >= 4 and l_quantity <= 4 + 10
                    and p_size between 1 and 5
                    and l_shipmode in ('AIR', 'AIR REG')
                    and l_shipinstruct = 'DELIVER IN PERSON'
                )
                or
                (
                    p_partkey = l_partkey
                    and p_brand = 'Brand#43'
                    and p_container in ('MED BAG', 'MED BOX', 'MED PKG', 'MED PACK')
                    and l_quantity >= 15 and l_quantity <= 25
                    and p_size between 1 and 10
                    and l_shipmode in ('AIR', 'AIR REG')
                    and l_shipinstruct = 'DELIVER IN PERSON'
                )
                or
                (
                    p_partkey = l_partkey
                    and p_brand = 'Brand#43'
                    and p_container in ('LG CASE', 'LG BOX', 'LG PACK', 'LG PKG')
                    and l_quantity >= 26 and l_quantity <= 36
                    and p_size between 1 and 15
                    and l_shipmode in ('AIR', 'AIR REG')
                    and l_shipinstruct = 'DELIVER IN PERSON'
                )"""
    )

    total.show()
    print("Q19 Execution time (s): ", time.time() - t1)


def q20():
    t1 = time.time()
    total = spark.sql(
        """select
                s_name,
                s_address
            from
                supplier,
                nation
            where
                s_suppkey in (
                    select
                        ps_suppkey
                    from
                        partsupp
                    where
                        ps_partkey in (
                            select
                                p_partkey
                            from
                                part
                            where
                                p_name like 'azure%'
                        )
                        and ps_availqty > (
                            select
                                0.5 * sum(l_quantity)
                            from
                                lineitem
                            where
                                l_partkey = ps_partkey
                                and l_suppkey = ps_suppkey
                                and l_shipdate >= date '1996-01-01'
                                and l_shipdate < date '1996-01-01' + interval '1' year
                        )
                )
                and s_nationkey = n_nationkey
                and n_name = 'JORDAN'
            order by
                s_name"""
    )

    total.show()
    print("Q20 Execution time (s): ", time.time() - t1)


def q21():
    t1 = time.time()
    NATION = "SAUDI ARABIA"
    total = spark.sql(
        f"""select
                s_name,
                count(*) as numwait
            from
                supplier,
                lineitem l1,
                orders,
                nation
            where
                s_suppkey = l1.l_suppkey
                and o_orderkey = l1.l_orderkey
                and o_orderstatus = 'F'
                and l1.l_receiptdate > l1.l_commitdate
                and exists (
                    select
                        *
                    from
                        lineitem l2
                    where
                        l2.l_orderkey = l1.l_orderkey
                        and l2.l_suppkey <> l1.l_suppkey
                )
                and not exists (
                    select
                        *
                    from
                        lineitem l3
                    where
                        l3.l_orderkey = l1.l_orderkey
                        and l3.l_suppkey <> l1.l_suppkey
                        and l3.l_receiptdate > l3.l_commitdate
                )
                and s_nationkey = n_nationkey
                and n_name = '{NATION}'
            group by
                s_name
            order by
                numwait desc,
                s_name"""
    )
    total.show()
    print("Q21 Execution time (s): ", time.time() - t1)



def q22():
    t1 = time.time()
    I1 = 13
    I2 = 31
    I3 = 23
    I4 = 29
    I5 = 30
    I6 = 18
    I7 = 17
    total = spark.sql(
        f"""select
                cntrycode,
                count(*) as numcust,
                sum(c_acctbal) as totacctbal
            from (
                select
                    substring(c_phone from 1 for 2) as cntrycode,
                    c_acctbal
                from
                    customer
                where
                    substring(c_phone from 1 for 2) in
                        ('{I1}','{I2}','{I3}','{I4}','{I5}','{I6}','{I7}')
                    and c_acctbal > (
                        select
                            avg(c_acctbal)
                        from
                            customer
                        where
                            c_acctbal > 0.00
                            and substring (c_phone from 1 for 2) in
                                ('{I1}','{I2}','{I3}','{I4}','{I5}','{I6}','{I7}')
                    )
                    and not exists (
                        select
                            *
                        from
                            orders
                        where
                            o_custkey = c_custkey
                    )
                ) as custsale
            group by
                cntrycode
            order by
                cntrycode"""
    )

    total.show()
    print("Q22 Execution time (s): ", time.time() - t1)


def run_queries():

    # Load the data
    t1 = time.time()

    lineitem = spark.table("nessie.SF1_LINEITEM_PQ_A")
    lineitem.createOrReplaceTempView("lineitem")

    part = spark.table("nessie.SF1_PART_PQ")
    part.createOrReplaceTempView("part")

    orders = spark.table("nessie.SF1_ORDERS_PQ")
    orders.createOrReplaceTempView("orders")

    customer = spark.table("nessie.SF1_CUSTOMER_PQ")
    customer.createOrReplaceTempView("customer")

    nation = spark.table("nessie.SF1_NATION_PQ")
    nation.createOrReplaceTempView("nation")

    region = spark.table("nessie.SF1_REGION_PQ")
    region.createOrReplaceTempView("region")

    supplier = spark.table("nessie.SF1_SUPPLIER_PQ")
    supplier.createOrReplaceTempView("supplier")

    partsupp = spark.table("nessie.SF1_PARTSUPP_PQ")
    partsupp.createOrReplaceTempView("partsupp")




    #print("Reading time (s): ", time.time() - t1)

    # Run the Queries:
    t1 = time.time()
    q01()
    q02()
    q03()
    q04()
    q05()
    q06()
    q07()
    q08()
    q09()
    q10()
    q11()
    q12()
    q13()
    q14()
    q15()
    q16()
    q17()
    q18()
    q19()
    q20()
    q21()
    q22()
    print("Total Query time (s): ", time.time() - t1)







VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
run_queries()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+-----------+--------------------+--------------------+--------------------+------------------+------------------+--------------------+-----------+
|l_returnflag|l_linestatus|    sum_qty|      sum_base_price|      sum_disc_price|          sum_charge|           avg_qty|         avg_price|            avg_disc|count_order|
+------------+------------+-----------+--------------------+--------------------+--------------------+------------------+------------------+--------------------+-----------+
|           A|           F|3.7734107E7|5.658655440072858E10|5.375825713487001E10|5.590906522282643E10|25.522005853257337| 38273.12973462071| 0.04998529583825147|    1478493|
|           N|           F|   991417.0| 1.487504710380006E9| 1.413082168054101E9|1.4696492231943767E9|25.516471920522985| 38284.46776084846|0.050093426674200345|      38854|
|           N|           O| 7.447604E7|1.117017296977359...|1.061182303076044...|1.103670438724958E11| 25.50222676958499| 38249.11